In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca
import time

category = 'all'

hNeg = True #if true, add negative bigrams for negative reviews
noun = True #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

In [2]:
X, new_vocabulary, index = pp.chi2(X, Y, vocabulary)

X = X.toarray()

In [3]:
def sigmoid(z):
    
    z = 1/(1+np.exp(-z))
    
    return z

In [4]:
def sigmoidGradient(z):

    g = np.zeros(z.shape)
    g = sigmoid(z)*(1 - sigmoid(z))    

    return g

In [5]:
def gridSearch(X, Y ,Xval, Yval):
    
#     lambda_rnn = [0.01, 0.1, 10, 100]
    
#     bestLambda = lambda_rnn[0]
#     bestAcc = 0
    
#     for l in lambda_rnn:
#         Theta1, Theta2 = redes_neurais(Xtrain, Ytrain, l)
#         Ypred, acuracia = predicao(Theta1, Theta2, Xval, Yval)
#         if bestAcc < acuracia :
#             bestLambda = l
#             bestAcc = acuracia

    bestLambda = 10
            
    return bestLambda

In [6]:
import scipy.optimize

def funcaoCusto_backp(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, flagreg = False,lambda_reg = 1):
    
    # Extrai os parametros de nn_params e alimenta as variaveis Theta1 e Theta2.
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )

    # Qtde de amostras
    m = X.shape[0]
         
    #Custo
    J = 0
    
    #Theta das redes neurais
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    eps = 1e-15

    
    rotulos = np.zeros((len(y), num_labels), dtype=int) 
    for i in range(len(rotulos)):
        rotulos[i][y[i]] = 1

    a1 = np.insert(X, 0, np.ones(m, dtype=int), axis=1)
    z2 = np.dot(a1, Theta1.T)
    a2 = sigmoid(z2)
    a2 = np.insert(a2, 0, np.ones(a2.shape[0], dtype=int), axis=1)
    z3 = np.dot(a2, Theta2.T)
    a3 = sigmoid(z3)
    
    
    delta3 = a3 - rotulos
    delta2 = np.dot(delta3, Theta2[:, 1:]) * sigmoidGradient(z2)

    if flagreg == True :
        reg = (lambda_reg/(2*m))*(np.sum(Theta1[:, 1:] ** 2) + np.sum(Theta2[:, 1:] ** 2)) 

        J += reg
    
        delta3 = a3 - rotulos
        delta2 = np.dot(delta3, Theta2[:, 1:]) * sigmoidGradient(z2)

        Theta1_grad[:, 0] = (np.dot(delta2.T, a1)[:, 0]/m) 
        Theta1_grad[:, 1:] = (np.dot(delta2.T, a1)[:, 1:]/m) + (lambda_reg/m)*Theta1[:, 1:]

        Theta2_grad[:, 0] = (np.dot(delta3.T, a2)[:, 0]/m) 
        Theta2_grad[:, 1:] = (np.dot(delta3.T, a2)[:, 1:]/m) + (lambda_reg/m)*Theta2[:, 1:]
    
    else:    
        Theta1_grad = np.dot(delta2.T, a1)/m
        Theta2_grad = np.dot(delta3.T, a2)/m
     
    J = np.sum(-rotulos*np.log(a3+eps) - (1 - rotulos)*np.log(1 - a3+eps))/m

    grad = np.concatenate([np.ravel(Theta1_grad), np.ravel(Theta2_grad)])

    return J, grad

In [7]:
def redes_neurais (Xtrain, Ytrain, lambda_rnn):

    input_layer_size  = Xtrain.shape[1]  # 20x20 dimensao das imagens de entrada
    hidden_layer_size = 25   
    num_labels = 2          # 10 rotulos, de 1 a 10  
                         #  (observe que a classe "0" recebe o rotulo 10)
    epsilon_init = 0.12
    
    # carregando os pesos da camada 1
    Theta1 =  np.random.RandomState(10).rand(hidden_layer_size, 1 + input_layer_size) * 2 * epsilon_init - epsilon_init
    
    # carregando os pesos da camada 2
    Theta2 = np.random.RandomState(10).rand(num_labels, 1 + hidden_layer_size) * 2 * epsilon_init - epsilon_init
    
    # concatena os pesos em um único vetor
    initial_rna_params = np.concatenate([np.ravel(Theta1), np.ravel(Theta2)])
    
    MaxIter = 5000
    
    #lamba_reg = 1
    
    # Minimiza a funcao de custo
    result = scipy.optimize.minimize(fun=funcaoCusto_backp, x0=initial_rna_params, args=(input_layer_size, hidden_layer_size, num_labels, Xtrain, Ytrain,True,lambda_rnn),  
                    method='TNC', jac=True, options={'maxiter': MaxIter})
    
    # Coleta os pesos retornados pela função de minimização
    nn_params = result.x
    
    # Obtem Theta1 e Theta2 back a partir de rna_params
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )
    
    return Theta1, Theta2

In [ ]:
def predicao(Theta1, Theta2, Xval, Yval):

    m = Xval.shape[0] # número de amostras
    num_labels = Theta2.shape[0]
    
    p = np.zeros(m)

    a1 = np.hstack( [np.ones([m,1]), Xval] )
    h1 = sigmoid( np.dot(a1, Theta1.T) )

    a2 = np.hstack( [np.ones([m,1]), h1] ) 
    h2 = sigmoid( np.dot(a2,Theta2.T) )
    
    Ypred = np.argmax(h2,axis=1)
    
    acuracia = np.sum(Ypred==Yval)/len(Yval) 
        
    return Ypred, acuracia

In [ ]:
lambda_rnn = gridSearch(X, Y, X, Y)
start_train = time.time()
Theta1, Theta2 = redes_neurais(X, Y, lambda_rnn)
end_train = time.time()
print("Tempo de treinamento é :" + str(end_train-start_train))

In [ ]:
with open('app/data/' + category + '/vocabulary.txt', 'w') as out_file:
    for word in new_vocabulary:
        if isinstance(word, tuple):
            s = '('+word[0]+","+word[1]+')'+'\n'
            out_file.write(s)
        else:
            out_file.write(word+'\n')

In [ ]:
np.save('app/data/' + category + '/Theta1', Theta1)
np.save('app/data/' + category + '/Theta2', Theta2)

In [ ]:
Ypred, acuracia = predicao(Theta1, Theta2, X, Y)
print(acuracia)